In [2]:
import os
import sys
import numpy as np
import pandas as pd
import openmatrix as omx

In [3]:
# Assume you have not changed the relative paths after cloning the repository
root = os.path.dirname(os.path.abspath('.'))
root

'D:\\src\\TransportationNetworks'

In [4]:
# We list all folders available, most of which are TNTP instances
folders = [x for x in os.listdir(root)[1:] if os.path.isdir(os.path.join(root, x))]

In [5]:
# Function to import OMX matrices
def import_matrix(matfile):
    f = open(matfile, 'r')
    all_rows = f.read()
    blocks = all_rows.split('Origin')[1:]
    matrix = {}
    for k in range(len(blocks)):
        orig = blocks[k].split('\n')
        dests = orig[1:]
        orig=int(orig[0])

        d = [eval('{'+a.replace(';',',').replace(' ','') +'}') for a in dests]
        destinations = {}
        for i in d:
            destinations = {**destinations, **i}
        matrix[orig] = destinations
    zones = max(matrix.keys())
    mat = np.zeros((zones, zones))
    for i in range(zones):
        for j in range(zones):
            # We map values to a index i-1, as Numpy is base 0
            mat[i, j] = matrix.get(i+1,{}).get(j+1,0)

    index = np.arange(zones) + 1

    myfile = omx.open_file('demand.omx','w')
    myfile['matrix'] = mat
    myfile.create_mapping('taz', index)
    myfile.close()

In [7]:
# If we want to import all matrices in place
for f in folders:
    mod = os.path.join(root, f)
    mod_files = os.listdir(mod)

    for i in mod_files:
        print(f.upper())
        if 'TRIPS' in i.upper() and i.lower()[-5:]=='.tntp':
            print('    trips')
            source_file = os.path.join(mod, i)
            import_matrix(source_file)

ANAHEIM
    trips
AUSTIN
    trips
BARCELONA
    trips
BERLIN-CENTER
    trips
BERLIN-FRIEDRICHSHAIN
    trips
BERLIN-MITTE-CENTER
    trips
BERLIN-MITTE-PRENZLAUERBERG-FRIEDRICHSHAIN-CENTER
    trips
BERLIN-PRENZLAUERBERG-CENTER
    trips
BERLIN-TIERGARTEN
    trips
BIRMINGHAM-ENGLAND
    trips
BRAESS-EXAMPLE
    trips
CHICAGO-SKETCH
    trips
EASTERN-MASSACHUSETTS
    trips
GOLDCOAST
    trips
HESSEN-ASYMMETRIC
    trips
PHILADELPHIA
    trips
SIOUXFALLS
    trips
SYDNEY
SYMMETRICATESTCASE
TERRASSA-ASYMMETRIC
    trips
WINNIPEG-ASYMMETRIC
    trips
WINNIPEG
    trips
CHICAGO-REGIONAL
    trips
_SCRIPTS


In [11]:
# Importing the networks into a Pandas dataframe consists of a single line of code
# but we can also make sure all headers are lower case and without trailing spaces

netfile = os.path.join(root, 'Anaheim','Anaheim_net.tntp')
net = pd.read_csv(netfile, skiprows=8, sep='\t')

trimmed= [s.strip().lower() for s in net.columns]
net.columns = trimmed

# And drop the silly first andlast columns
net.drop(['~', ';'], axis=1, inplace=True)

In [24]:
net.head()

,init_node,term_node,capacity,length,free_flow_time,b,power,speed,toll,link_type,;
0,1,117,9000,5280,1.090458,0.15,4,4842,0,1,;
1,2,87,9000,5280,1.090458,0.15,4,4842,0,1,;
2,3,74,9000,5280,1.090458,0.15,4,4842,0,1,;
3,4,233,9000,5280,1.090458,0.15,4,4842,0,1,;
4,5,165,9000,5280,1.090458,0.15,4,4842,0,1,;


In [14]:
net.columns

Index(['init_node', 'term_node', 'capacity', 'length', 'free_flow_time', 'b',
       'power', 'speed', 'toll', 'link_type'],
      dtype='object')